In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

# Check if GPU is available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

TensorFlow version: 2.17.0
GPU is available


In [ ]:
!ls '/content/drive/My Drive/'
import zipfile


 1699556619213.jpg
 1699556619225.jpg
'1 Million Subscribers Gift - BMBULLS'
 22CH10084_Assign_1_TC.pdf
 22CH10084_practice_sheet_TC.pdf
 aagaz
'American Expres Campus Challenge 2024'
'AMEX features doc.gdoc'
 Answer_doc.gdoc
 assign5_22CH10084.pdf
'batsman level.gsheet'
'CDC Intern 2023'
'Colab Notebooks'
 Cold_mailing_phand.gsheet
'Copy of AHCM-HealthSocialNeedsScreeningTool-rev-8_10_2021.pdf'
'Copy of Gmail Sheets mail merge  (10).gsheet'
'Copy of Gmail Sheets mail merge  (11).gsheet'
'Copy of Gmail Sheets mail merge  (12).gsheet'
'Copy of Gmail Sheets mail merge  (13).gsheet'
'Copy of Gmail Sheets mail merge  (14).gsheet'
'Copy of Gmail Sheets mail merge  (15).gsheet'
'Copy of Gmail Sheets mail merge  (16).gsheet'
'Copy of Gmail Sheets mail merge  (17).gsheet'
'Copy of Gmail Sheets mail merge  (18).gsheet'
'Copy of Gmail Sheets mail merge  (19).gsheet'
'Copy of Gmail Sheets mail merge  (1).gsheet'
'Copy of Gmail Sheets mail merge  (20).gsheet'
'Copy of Gmail Sheets mail merge  (21)

In [ ]:

import zipfile

# Path to the zipped folders
zip_file1 = '/content/drive/My Drive/ntu_frames2.zip'
zip_file2 = '/content/drive/My Drive/ntu_frames.zip'
# Extract folder1
with zipfile.ZipFile(zip_file1, 'r') as zip_ref:
    zip_ref.extractall('/content/folder1')

# Extract folder1
with zipfile.ZipFile(zip_file2, 'r') as zip_ref:
    zip_ref.extractall('/content/folder1')


In [ ]:
zip_file3= '/content/drive/My Drive/csv_zip.zip'
with zipfile.ZipFile(zip_file3, 'r') as zip_ref:
    zip_ref.extractall('/content/folder3')


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
def load_frames_and_ratings(data_folders, csv_folder, target_size=(64, 64), num_frames=5):
    data = []
    all_ratings = []

    for data_folder in data_folders:
        subfolders = [f for f in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, f))]

        for subfolder in subfolders:
            subfolder_path = os.path.join(data_folder, subfolder)
            csv_file_name = '_'.join(subfolder.split('_')[1:3]) + '_VideoNumberStartEndRatings.csv'
            csv_file = os.path.join(csv_folder, csv_file_name)

            if os.path.exists(csv_file):
                ratings_df = pd.read_csv(csv_file)

                for frames_folder_name in os.listdir(subfolder_path):
                    frames_folder_path = os.path.join(subfolder_path, frames_folder_name)
                    frame_files = sorted([f for f in os.listdir(frames_folder_path) if f.endswith('.jpg')])
                    video_num = int(frames_folder_name.split('_')[-1])
                    video_num_padded = str(video_num).zfill(2)
                    row = ratings_df[ratings_df['Videonum'] == video_num_padded]

                    if not row.empty:
                        rating = [
                            row['surpriseRate'].values[0],
                            row['curiosityRate'].values[0],
                            row['confusionRate'].values[0]
                        ]

                        # Calculate the subsample rate to get exactly num_frames frames
                        subsample_rate = max(len(frame_files) // num_frames, 1)
                        selected_frames = []

                        for i, filename in enumerate(frame_files):
                            if i % subsample_rate == 0:
                                img = load_img(os.path.join(frames_folder_path, filename), target_size=target_size)
                                img_array = img_to_array(img) / 255.0  # Convert to RGB and normalize
                                selected_frames.append(img_array)
                            if len(selected_frames) == num_frames:
                                break

                        # If the number of selected frames is less than num_frames, pad with zeros
                        while len(selected_frames) < num_frames:
                            selected_frames.append(np.zeros((target_size[0], target_size[1], 3)))

                        # Stack arrays to ensure homogeneous shape
                        if selected_frames:
                            data.append(np.stack(selected_frames))  # Stack to ensure homogeneous shape
                            all_ratings.append(np.array(rating))

    return np.array(data), np.array(all_ratings)

In [ ]:
data_folders = ['/content/folder1/ntu_frames','/content/folder1/ntu_frames2']
csv_folder = '/content/folder3/csv'
frames_data, ratings_data = load_frames_and_ratings(data_folders, csv_folder)
from sklearn.model_selection import train_test_split
# Split data into train, validation, and test sets
train_frames, test_frames, train_ratings, test_ratings = train_test_split(frames_data, ratings_data, test_size=0.2, random_state=42)
train_frames, val_frames, train_ratings, val_ratings = train_test_split(train_frames, train_ratings, test_size=0.2, random_state=42)

print(f'Train frames shape: {train_frames.shape}')
print(f'Train ratings shape: {train_ratings.shape}')
print(f'Validation frames shape: {val_frames.shape}')
print(f'Validation ratings shape: {val_ratings.shape}')
print(f'Test frames shape: {test_frames.shape}')
print(f'Test ratings shape: {test_ratings.shape}')

Train frames shape: (628, 5, 64, 64, 3)
Train ratings shape: (628, 3)
Validation frames shape: (158, 5, 64, 64, 3)
Validation ratings shape: (158, 3)
Test frames shape: (197, 5, 64, 64, 3)
Test ratings shape: (197, 3)


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
import tensorflow as tf

# Configure TensorFlow to use the GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("GPU is available and configured for use")
else:
    print("No GPU available. Using CPU.")

GPU is available and configured for use


In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [ ]:
frame_input = Input(shape=(5, 64, 64, 3), name='frame_input')

In [ ]:
'''
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Layer, Conv3D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16

# Enable mixed precision training
from tensorflow.keras.mixed_precision import global_policy, set_global_policy, Policy
policy = Policy('mixed_float16')
set_global_policy(policy)

class CustomVGG16(Layer):
    def __init__(self):
        super(CustomVGG16, self).__init__()
        self.vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

    def call(self, inputs):
        return self.vgg16(inputs)

    def compute_output_shape(self, input_shape):
        return self.vgg16.output_shape

# Apply the CustomVGG16 layer
vgg16_features = CustomVGG16()(frame_input)

# Flatten the output from VGG16
flattened_features = tf.keras.layers.Flatten()(vgg16_features)
x = LSTM(64, return_sequences=False)(flattened_features)
# Add Dense layers
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Define the outputs
surprise_output = Dense(1, activation='linear', name='surprise_output')(x)
curiosity_output = Dense(1, activation='linear', name='curiosity_output')(x)
confusion_output = Dense(1, activation='linear', name='confusion_output')(x)

# Create the model
model = Model(inputs=frame_input, outputs=[surprise_output, curiosity_output, confusion_output])

with tf.device('/GPU:0'):
    model.compile(optimizer='adam',
                  loss={'surprise_output': 'mse',
                        'curiosity_output': 'mse',
                        'confusion_output': 'mse'},
                  loss_weights={'surprise_output': 1.0,
                                'curiosity_output': 1.0,
                                'confusion_output': 1.0})
# Print model summary
model.summary()


In [ ]:
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, Flatten
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.applications import vgg19
from tensorflow.keras.layers import Bidirectional, Dropout

# Define frame input with shape (num_frames, height, width, channels)
frame_input = Input(shape=(5, 64, 64, 3), name='frame_input')

# Use ResNet50 without the top classification layer
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Apply ResNet50 to each frame using TimeDistributed
cnn_features = TimeDistributed(resnet_model)(frame_input)

# Step 2: Flatten the output from MobileNetV2
flattened_features = TimeDistributed(Flatten())(cnn_features)

# Step 3: Use a GRU layer to model temporal dependencies across frames
x =LSTM(64, return_sequences=False)(flattened_features)

x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)

# Step 5: Define the output layers for surprise, curiosity, and confusion
surprise_output = Dense(1, activation='linear', name='surprise_output')(x)
curiosity_output = Dense(1, activation='linear', name='curiosity_output')(x)
confusion_output = Dense(1, activation='linear', name='confusion_output')(x)

# Create the model
model = Model(inputs=frame_input, outputs=[surprise_output, curiosity_output, confusion_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'surprise_output': 'mse',
                    'curiosity_output': 'mse',
                    'confusion_output': 'mse'})
model.summary()

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_34 (InputLayer)          │ (None, 5, 64, 64, 3)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_14                  │ (None, 5, 2, 2, 512)        │      20,024,384 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_15                  │ (None, 5, 512)              │               0 │
│ (TimeDistributed)                    │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 2560)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 128)                 │         327,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_44 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,352,579 (77.64 MB)

 Trainable params: 20,352,579 (77.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import matplotlib.pyplot as plt

# Train the model and capture the history
with tf.device('/GPU:0'):
    history = model.fit(train_frames,
                        [train_ratings[:, 0], train_ratings[:, 1], train_ratings[:, 2]],
                        batch_size=32,
                        epochs=20,
                        validation_data=(val_frames, [val_ratings[:, 0], val_ratings[:, 1], val_ratings[:, 2]]))

# Extract training and validation loss from history
train_loss = history.history['loss']  # Overall training loss
val_loss = history.history['val_loss']  # Overall validation loss

# Plot the training and validation loss over epochs
plt.figure(figsize=(10, 6))
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Overall Training and Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/mse/sub defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [32] vs. [32,3]
	 [[{{node compile_loss/mse/sub}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_662379[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_662612]

In [ ]:
# Convert predictions and true ratings to binary based on the threshold
threshold = 3
surprise_pred_binary = (surprise_pred > threshold).astype(int)
curiosity_pred_binary = (curiosity_pred > threshold).astype(int)
confusion_pred_binary = (confusion_pred > threshold).astype(int)

surprise_true_binary = (test_ratings[:, 0] > threshold).astype(int)
curiosity_true_binary = (test_ratings[:, 1] > threshold).astype(int)
confusion_true_binary = (test_ratings[:, 2] > threshold).astype(int)

# Calculate metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

surprise_precision = precision_score(surprise_true_binary, surprise_pred_binary, average='macro')
surprise_recall = recall_score(surprise_true_binary, surprise_pred_binary, average='macro')
surprise_accuracy = accuracy_score(surprise_true_binary, surprise_pred_binary)

curiosity_precision = precision_score(curiosity_true_binary, curiosity_pred_binary, average='macro')
curiosity_recall = recall_score(curiosity_true_binary, curiosity_pred_binary, average='macro')
curiosity_accuracy = accuracy_score(curiosity_true_binary, curiosity_pred_binary)

confusion_precision = precision_score(confusion_true_binary, confusion_pred_binary, average='macro')
confusion_recall = recall_score(confusion_true_binary, confusion_pred_binary, average='macro')
confusion_accuracy = accuracy_score(confusion_true_binary, confusion_pred_binary)

# Print the metrics
print(f"Surprise - Precision: {surprise_precision}, Recall: {surprise_recall}, Accuracy: {surprise_accuracy}")
print(f"Curiosity - Precision: {curiosity_precision}, Recall: {curiosity_recall}, Accuracy: {curiosity_accuracy}")
print(f"Confusion - Precision: {confusion_precision}, Recall: {confusion_recall}, Accuracy: {confusion_accuracy}")

Surprise - Precision: 0.5808854831843338, Recall: 0.5805938494167551, Accuracy: 0.5634517766497462
Curiosity - Precision: 0.6352398910538446, Recall: 0.635609243697479, Accuracy: 0.6243654822335025
Confusion - Precision: 0.5992638980443858, Recall: 0.5987431693989071, Accuracy: 0.5634517766497462
